In [2]:
#file to determine the cost for a specfic or several molecules from custom cost calculator

In [1]:
import os
import sys
import argparse

import json
import pandas as pd
import numpy as np

sys.path.append('../../')

import Maranga.utils.parse_multismiles as mutils
import Maranga.scripts.fingerprints as fings
import aizynthfinder.context.scoring as scoring
import aizynthfinder.context.config as con

from aizynthfinder.analysis import ReactionTree
from rdkit.Chem import rdChemReactions
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import euclidean

In [49]:
# read in file and generate list of molecules from first 20
smiles = []
with open('/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/explore/data/tg1_smiles.txt') as file:
    firstNlines=file.readlines()[0:20]
smiles = [i.strip('\n') for i in firstNlines]

In [2]:
config_file = '/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/aizynthfinder/data/config.yml'

In [73]:
# import data from file
data = mutils.read_hdf('/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/explore/results/tg1_explore_3.hdf5')

In [74]:
# filter reactions for specific molecule
molecule_data = data.loc[(data.target=='O=S(=O)(NCC(c1ccsc1)N1CCCCCC1)c1cccs1')]
trees = mutils.collect_trees(molecule_data)
rxn_trees = fings.parse_input(trees, 'explore')
pathways = [i.get('reaction') for i in rxn_trees]

Multiple Molecules


In [75]:
# calculate cost for each route
cost = mutils.calculate_cost_from_trees(pathways, config_file)
print('Mean cost: ', np.mean(cost))
print('Mean cost: ', np.std(cost))

Reaction:  0
Policy:  0.3751999978533907
Precursors:  40.0
Score:  106.60980871228573
/n
Reaction:  1
Policy:  0.42408889295943986
Precursors:  70.0
Score:  165.05973432012246
/n
Reaction:  2
Policy:  0.6384999930858613
Precursors:  50.0
Score:  78.30853647836506
/n
Reaction:  3
Policy:  0.5976333364160382
Precursors:  40.0
Score:  66.93067063473562
/n
Reaction:  4
Policy:  0.4719600041513331
Precursors:  40.0
Score:  84.75294441936244
/n
Mean cost:  100.33233891297427
Mean cost:  34.84961748286587


In [43]:
# determine the reaction classifications used
classes = [list(mutils.findkeys(i, 'classification')) for i in pathways]

#flatten list
class_ = []
for i in classes:
    class_.extend(i)
    
# convert to frequency dict    
reaction_dict = {}
for i in class_:
    if i in reaction_dict:
        reaction_dict[i] += 1
    else:
        reaction_dict[i] = 1


In [44]:
print(reaction_dict)

{'Amide to amine reduction': 1, 'N-acylation to amide': 2, 'Unassigned': 1, 'Alkene to alkane': 1, 'N-sulfonylation': 4, 'Nitro to amine reduction': 2, 'Other organometallic C-C bond formation': 2, 'Heteroaryl N-alkylation': 2, 'Reductive amination': 3, 'Sulfonation': 4, 'Other C-C bond formation': 1, 'Cyano or imine to amine': 1, 'Halogenation': 1}


In [60]:
json_files = '/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/explore/results/tg1_393_opt2.json'

In [65]:
with open(json_files, 'r') as input:
        data = json.load(input)

In [68]:
print(len(data))

5
